In [1]:
from huggingface_hub import HfApi
from huggingface_hub import upload_file, delete_file,upload_folder

import re
import torch
import json
import pandas as pd
import numpy as np

from torch.utils.data import DataLoader
from datasets import load_dataset
from sentence_transformers import SentenceTransformer,  LoggingHandler, losses, models, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample
from transformers import AutoModel, AutoTokenizer
from top2vec import Top2Vec


In [7]:
embedding_model = models.Transformer(
    model_name_or_path="KDHyun08/TAACO_STS", 
    max_seq_length=256,
    do_lower_case=True
)

pooling_model = models.Pooling(
    embedding_model.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=True,
    pooling_mode_cls_token=False,
    pooling_mode_max_tokens=False,
)
model = SentenceTransformer(modules=[embedding_model, pooling_model])

In [8]:
highschool=0.0

In [9]:
docs = ['어제는 아내의 생일이었다', '생일을 맞이하여 아침을 준비하겠다고 오전 8시 30분부터 음식을 준비하였다. 주된 메뉴는 스테이크와 낙지볶음, 미역국, 잡채, 소야 등이었다', '스테이크는 자주 하는 음식이어서 자신이 준비하려고 했다', '앞뒤도 1분씩 3번 뒤집고 래스팅을 잘 하면 육즙이 가득한 스테이크가 준비되다', '아내도 그런 스테이크를 좋아한다. 그런데 상상도 못한 일이 벌이지고 말았다', '보통 시즈닝이 되지 않은 원육을 사서 스테이크를 했는데, 이번에는 시즈닝이 된 부챗살을 구입해서 했다', '그런데 케이스 안에 방부제가 들어있는 것을 인지하지 못하고 방부제와 동시에 프라이팬에 올려놓을 것이다', '그것도 인지 못한 체... 앞면을 센 불에 1분을 굽고 뒤집는 순간 방부제가 함께 구어진 것을 알았다', '아내의 생일이라 맛있게 구워보고 싶었는데 어처구니없는 상황이 발생한 것이다', '방부제가 센 불에 녹아서 그런지 물처럼 흘러내렸다', ' 고민을 했다. 방부제가 묻은 부문만 제거하고 다시 구울까 했는데 방부제에 절대 먹지 말라는 문구가 있어서 아깝지만 버리는 방향을 했다', '너무나 안타까웠다', '아침 일찍 아내가 좋아하는 스테이크를 준비하고 그것을 맛있게 먹는 아내의 모습을 보고 싶었는데 전혀 생각지도 못한 상황이 발생해서... 하지만 정신을 추스르고 바로 다른 메뉴로 변경했다', '소야, 소시지 야채볶음..', '아내가 좋아하는지 모르겠지만 냉장고 안에 있는 후랑크소세지를 보니 바로 소야를 해야겠다는 생각이 들었다. 음식은 성공적으로 완성이 되었다', '40번째를 맞이하는 아내의 생일은 성공적으로 준비가 되었다', '맛있게 먹어 준 아내에게도 감사했다', '매년 아내의 생일에 맞이하면 아침마다 생일을 차려야겠다. 오늘도 즐거운 하루가 되었으면 좋겠다', '생일이니까~']
text="놀이. 대부분의 전래 놀이는 바깥에서 한다. 옛날 아이들은 윷놀이, 자치기, 팽이치기와 같은 전래 놀이를 하면서 친구들과 즐겁게 놀았다. 요즘 아이들은 주로 인터넷 게임을 하거나 텔레비전을 시청하며 여가를 보낸다. 전래 놀이를 하면 다음과 같은 좋은 점이 있다.첫째, 자연을 가까이 느낄 수 있다. 대부분의 전래 놀이는 바깥에서 한다. 그래서 풀, 바람, 나무, 흙, 돌, 햇빛 등 자연을 가까이 느낄 수 있다. 둘째, 공동체 의식을 기를 수 있다. 또래 아이들과 함께 윷놀이나 비사치기를 하면서 의논하고 힘을 모으기도 한다. 이러한 과정을 거치면서 아이들은 서로 돕고 위하는 태도를 배우게 된다. 이와 같이 전래 놀이를 하면 좋은 점이 많다. 우리 모두 전래 놀이에 관심을 가지고 즐겨 하자."
docs=re.split('\. |\? |\!',text)

#각 문장의 vector값 encoding
document_embeddings = model.encode(docs)

query = docs[0]
query_embedding = model.encode(query)

top_k = len(docs)
score_sum=0.0
# 코사인 유사도 계산 후,
cos_scores = util.pytorch_cos_sim(query_embedding, document_embeddings)[0]

# 코사인 유사도 순으로 문장 추출
top_results = torch.topk(cos_scores, k=top_k)

print(f"입력 문장: {query}")
print(f"\n<입력 문장과 유사한 {top_k} 개의 문장>\n")

for i, (score, idx) in enumerate(zip(top_results[0], top_results[1])):
    score_sum+=score
    print(f"{i+1}: {docs[idx]} {'(유사도: {:.4f})'.format(score)}\n")
highschool+=(score_sum-1)/(top_k-1)
print(highschool)

입력 문장: 놀이

<입력 문장과 유사한 12 개의 문장>

1: 놀이 (유사도: 1.0000)

2: 대부분의 전래 놀이는 바깥에서 한다 (유사도: 0.4440)

3: 대부분의 전래 놀이는 바깥에서 한다 (유사도: 0.4440)

4: 우리 모두 전래 놀이에 관심을 가지고 즐겨 하자. (유사도: 0.3809)

5: 이와 같이 전래 놀이를 하면 좋은 점이 많다 (유사도: 0.3220)

6: 전래 놀이를 하면 다음과 같은 좋은 점이 있다.첫째, 자연을 가까이 느낄 수 있다 (유사도: 0.3001)

7: 옛날 아이들은 윷놀이, 자치기, 팽이치기와 같은 전래 놀이를 하면서 친구들과 즐겁게 놀았다 (유사도: 0.2954)

8: 요즘 아이들은 주로 인터넷 게임을 하거나 텔레비전을 시청하며 여가를 보낸다 (유사도: 0.2535)

9: 또래 아이들과 함께 윷놀이나 비사치기를 하면서 의논하고 힘을 모으기도 한다 (유사도: 0.2410)

10: 그래서 풀, 바람, 나무, 흙, 돌, 햇빛 등 자연을 가까이 느낄 수 있다 (유사도: 0.1256)

11: 이러한 과정을 거치면서 아이들은 서로 돕고 위하는 태도를 배우게 된다 (유사도: 0.0991)

12: 둘째, 공동체 의식을 기를 수 있다 (유사도: -0.0165)

tensor(0.2627)


In [11]:
#Top2Vec 모델
data=pd.read_csv("data/test.csv")
text=list()
text=data['text'].to_list()
model2=Top2Vec(documents=list(text),speed='fast-learn',workers=8,embedding_model='distiluse-base-multilingual-cased')

2023-01-12 11:25:34,533 - top2vec - INFO - Pre-processing documents for training
C:\Users\DESKTOP\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
2023-01-12 11:27:01,507 - top2vec - INFO - Downloading distiluse-base-multilingual-cased model
2023-01-12 11:27:02,941 - top2vec - INFO - Creating joint document/word embedding
2023-01-12 15:28:45,029 - top2vec - INFO - Creating lower dimension embedding of documents
2023-01-12 15:31:22,572 - top2vec - INFO - Finding dense areas of documents
2023-01-12 15:31:43,171 - top2vec - INFO - Finding topics


In [14]:
topic_words, word_scores, topic_nums = model2.get_topics(1)
topic_words

array([['영화산업이', '영화산업의', '영화산업에', '영화산업', '영화계의', '한국영화는', '한국영화를',
        '한국영화의', '한국영화가', '영화제작에', '영화들은', '한국영화', '영화콘텐츠의', '영화에서',
        '영화들을', '필름을', '영화들의', '영화제작', '한국영화에', '영화계', '일본영화', '영화들이',
        '영화는', '필름의', '영화진흥위원회의', '극장에', '영화관', '영화진흥위원회', '영화를', '영화인',
        'film', '극장에서', '극영화', '필름', '영화의', '자국영화', '독립영화', '극장을', '영화제',
        '극장이', '극장용', '영화와', '할리우드의', '영화가', '극장', '한국산업인력공단', '극장은',
        '영상물을', '영화', '예술영화']], dtype='<U15')

In [4]:
print(model2.get_documents_topics([0])[3][0])

[[0.5724424  0.52618325 0.52609956 0.5226298  0.5209547  0.51504314
  0.51408887 0.51052535 0.49913186 0.44753253 0.43170786 0.35782516
  0.3172083  0.31363225 0.30822423 0.30444226 0.29602402 0.28985986
  0.28927082 0.27344716 0.26482677 0.2563244  0.25419402 0.25280416
  0.25232914 0.25083512 0.2444829  0.23860112 0.23154777 0.23070747
  0.22920698 0.2275315  0.21959925 0.21580896 0.21460935 0.21348289
  0.21343178 0.212675   0.21184263 0.21048611 0.20994973 0.20991236
  0.20986226 0.20672822 0.20572194 0.20547816 0.20131412 0.19975272
  0.19876665 0.19852097]]


In [ ]:
idx=340391
sum=0.0
cnt=80
for i in range(0,cnt):
    try:
        docutopi=model2.get_documents_topics([idx+i])[3][0][0]
        sum+=docutopi
    except:
        break
sum=sum/cnt
print(sum)
